# Session 会话

In [ ]:
import tensorflow as tf
import numpy as np


# create data
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data*0.1 + 0.3

#create tnesorflow struvture start #
Weights = tf.Variable(tf.random_uniform([1],-1,1))
biases = tf.Variable(tf.zeros([1]))

y = Weights*x_data +biases

loss = tf.reduce_mean(tf.square(y-y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
#create tnesorflow struvture end #

sess = tf.Session()
sess.run(init) #improtant

for step in range(201):
    sess.run(train)
    if step % 20 ==0:
        print(step, sess.run(Weights),sess.run(biases))

In [ ]:
import tensorflow as tf
matrix1 = tf.constant([[3,3]])
matrix2 = tf.constant([[2],
                       [2]])
product = tf.matmul(matrix1,matrix2)  #matrix multiply np.dot(m1,m2)
# #method 1
# sess = tf.Session()
# result = sess.run(product)
# print(result)
# sess.close()

#method 2
with tf.Session() as sess:
    result2 = sess.run(product)
    print(result2)
    

# 变量

In [ ]:
import tensorflow as tf
state = tf.Variable(0,name='counter')
# print(state.name)
one = tf.constant(1)

new_value = tf.add(state,one)
update = tf.assign(state,new_value)

init = tf.initialize_all_variables()#must have if difne variables

with tf.Session() as sess:
    sess.run(init)
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))
    

# 传入值

In [ ]:
import tensorflow as tf
#input1 = tf.palceholder(tf.float32,[2,2])
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1,input2)
with tf.Session() as sess:
    print(sess.run(output,feed_dict = {input1:[7.],input2:[2.]}))

# 10 构建添加层

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


#%matplotlib inline

def add_layer(inputs,in_size,out_size,activation_function = None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]),name = 'W')
    biases = tf.Variable(tf.zeros([1,out_size])+0.1,name = 'b')
    Wx_puls_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_puls_b 
    else:
        outputs = activation_function(Wx_puls_b,)
    return outputs

x_data = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0,0.05,x_data.shape)
y_data = np.square(x_data)-0.5+noise

xs = tf.placeholder(tf.float32,[None,1],name='x_input')
ys = tf.placeholder(tf.float32,[None,1],name = 'y_input')

layer1 = add_layer(xs,1,10,activation_function = tf.nn.relu)
prediction = add_layer(layer1,10,1,activation_function = None)


loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                    reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
sess = tf.Session()

sess.run(init)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(x_data,y_data)
plt.ion()
#plt.show()
#plt.show(block = False)


for step in range(1000):
    sess.run(train_step,feed_dict={xs:x_data,ys:y_data})
    if step%50==0:
        print(step,sess.run(loss,feed_dict={xs:x_data,ys:y_data}))
        try:
            ax.line.remove(lines[0])
        except Exception:
            pass
        prediction_value = sess.run(prediction,feed_dict={xs:x_data})
        
        #plt.pause(0.1)
        #plt.close()
lines = ax.plot(x_data, prediction_value, 'r-', lw=5)
# plt.scatter(x_data, prediction_value)
# plt.pause(0.05)       


# 14 tensorboard graph

In [ ]:
# tensorboard --logdir=C://Users//Administrator//logs


"""
Please note, this code is only for python 3+. If you are using python 2+, please modify the code accordingly.
"""
import tensorflow as tf


def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        return outputs


# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input') 
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')
 

# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()
#writer = tf.train.SummaryWriter("logs/", sess.graph)
writer = tf.summary.FileWriter("logs/",sess.graph)
# important step
sess.run(tf.initialize_all_variables())




# 15 tensorboard histograms

In [1]:
 

"""
Please note, this code is only for python 3+. If you are using python 2+, please modify the code accordingly.
"""
import tensorflow as tf
import numpy as np


def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            #tf.histogram_summary(layer_name + '/weights', Weights)
            tf.summary.histogram(layer_name + '/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        tf.summary.histogram(layer_name + '/outputs', outputs)
        return outputs


# Make up some real data
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 1, 10, n_layer=1, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, n_layer=2, activation_function=None)

# the error between prediciton and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                                        reduction_indices=[1]))
    #tf.scalar_summary('loss', loss)
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

sess = tf.Session()
#merged = tf.merge_all_summaries()
merged = tf.summary.merge_all()
#writer = tf.train.SummaryWriter("logs/", sess.graph)
writer = tf.summary.FileWriter("logs/", sess.graph)
# important step
sess.run(tf.global_variables_initializer())

for i in range(1000):
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        result = sess.run(merged,
                          feed_dict={xs: x_data, ys: y_data})
        writer.add_summary(result, i)


D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 16 Classification

In [ ]:
import tensorflow as tf  

from tensorflow.examples.tutorials.mnist import input_data  
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)  

def add_layer(inputs,in_size,out_size,activation_function = None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]),name = 'W')
    biases = tf.Variable(tf.zeros([1,out_size])+0.1,name = 'b')
    Wx_puls_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_puls_b 
    else:
        outputs = activation_function(Wx_puls_b,)
    return outputs

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pre = sess.run(prediction,feed_dict={x:v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={x:v_xs,y_:v_ys})
    return result

#define placeholder for inputs to netword
x = tf.placeholder("float", [None, 784])  #28*28
y_ = tf.placeholder("float", [None,10])

#add ouyput layer
prediction = add_layer(x,784,10,activation_function = tf.nn.softmax)

#the error between prediction and real data 计算交叉熵:
#cross_entropy = -tf.reduce_sum(y_*tf.log(y))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(prediction),
                                              reduction_indices=[1]))#loss
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i%50 == 0:  
#         correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#         accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#         print ("step %d, training accuracy %g"%(i, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))  
          print(i,compute_accuracy(mnist.test.images,mnist.test.labels))
# saver = tf.train.Saver()   
# saver.save(sess, "model_data/model.ckpt")  

# 19 Saver

In [21]:
import tensorflow as tf  
import numpy as np

#Save to file
#remember to define the same dtype and shape when restore
W = tf.Variable([[3,4,5],[6,7,8]],dtype = tf.float32,name = 'weights')
b = tf.Variable([[3,6,9]],dtype = tf.float32,name = 'biases')

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    save_path = saver.save(sess,"my_net/save_net.ckpt")
    print("Save to path:",save_path)
    
# #restore variables
# #redefine the same dtype and shape when restore
# W = tf.Variable(np.arange(6).reshape((2,3)),dtype = tf.float32,name = "weights")
# b = tf.Variable(np.arange(3).reshape((1,3)),dtype = tf.float32,name = "biases")

# #not need init step
# saver = tf.train.Saver()
# with tf.Session() as sess:
#     saver.restore(sess,"my_net/save_net.ckpt")
#     print("weights:",sess.run(W))
#     print("biases:",sess.run(b))

Save to path: my_net/save_net.ckpt


#  读取图片和模型教程

In [ ]:
from PIL import Image, ImageFilter
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

def imageprepare():
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    file_name='images/3.png'#导入自己的图片地址
    #in terminal 'mogrify -format png *.jpg' convert jpg to png
    im = Image.open(file_name).convert('L')

    im.save("images/sample.png")
    plt.imshow(im)
    plt.show()
    tv = list(im.getdata()) #get pixel values

    #normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [ (255-x)*1.0/255.0 for x in tv] 
    #print(tva)
    return tva

"""
This function returns the predicted integer.
The imput is the pixel values from the imageprepare() function.
"""

# Define the model (same as when creating the model file)
result=imageprepare()

#定义一些函数：分配系数函数、分配偏置函数、卷积函数、pooling函数  
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')   

#x是输入的图像，y_是对应的标签  
x = tf.placeholder(tf.float32, [None, 784])  
y_ = tf.placeholder(tf.float32, [None, 10])  

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
#第1层卷积层，Receptive Field 5＊5，单个batch生成32通道数据  
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
#把图像向量还原成28＊28的图像  
x_image = tf.reshape(x, [-1,28,28,1])  
#第1个卷积层，使用了ReLU激活函数  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)   
#第2层卷积层，Receptive Field 5＊5，单个batch 32通道生成64通道数据  
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
#第2个卷积层  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
#全链接层系数  
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
#全链接层：把64通道数据展开方便全链接  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
#全链层神经元使用dropout防止过拟合  
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  
#softmax层系数  
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])  
#softmax层  
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  


#初始化
#sess = tf.InteractiveSession()
#init_op = tf.initialize_all_variables()
init = tf.global_variables_initializer()  #新的方式
#sess.run(tf.global_variables_initializer())  
"""
Load the model2.ckpt file
file is stored in the same directory as this python script is started
Use the model to predict the integer. Integer is returend as list.

Based on the documentatoin at
https://www.tensorflow.org/versions/master/how_tos/variables/index.html
"""
saver = tf.train.Saver()
#saver.save(sess, "model_data/model.ckpt")  #训练完用来保存
saver.restore(sess, 'model_data/model.ckpt') #调用
with tf.Session() as sess:
    sess.run(init) #初始化
    saver.restore(sess, 'model_data/model.ckpt')#这里使用了之前保存的模型参数
    #print ("Model restored.")

    prediction=tf.argmax(y_conv,1)
    predint=prediction.eval(feed_dict={x: [result],keep_prob: 1.0}, session=sess)
    print(h_conv2)

    print('recognize result:')
    print(predint[0])